# **RAG @ `typesense`**

In [1]:
import typesense

In [ ]:
client = typesense.Client(
    {
        'nodes': [{
            'host': '7k2vp1z3qrhta4x5p-1.a1.typesense.net',
            'port': '443',
            'protocol': 'https',
        }],
        'api_key': 'YOUR_TYPESENSE_API_KEY', # The Admin API (and not the READ-ONLY API)
        'connection_timout_seconds': 2
    }
)

In [3]:
client

In [4]:
# Lets create teh schema

books_schema = {
  'name': 'books',
  'fields': [
    {'name': 'title', 'type': 'string'},
    {'name': 'authors', 'type': 'string[]', 'facet': True},
    {'name': 'publication_year', 'type': 'int32', 'facet': True},
    {'name': 'ratings_count', 'type': 'int32'},
    {'name': 'average_rating', 'type': 'float'}
  ],
  'default_sorting_field': 'ratings_count'
}
print(client.collections.create(books_schema))

{'created_at': 1759417741, 'default_sorting_field': 'ratings_count', 'enable_nested_fields': False, 'fields': [{'facet': False, 'index': True, 'infix': False, 'locale': '', 'name': 'title', 'optional': False, 'sort': False, 'stem': False, 'stem_dictionary': '', 'store': True, 'type': 'string'}, {'facet': True, 'index': True, 'infix': False, 'locale': '', 'name': 'authors', 'optional': False, 'sort': False, 'stem': False, 'stem_dictionary': '', 'store': True, 'type': 'string[]'}, {'facet': True, 'index': True, 'infix': False, 'locale': '', 'name': 'publication_year', 'optional': False, 'sort': True, 'stem': False, 'stem_dictionary': '', 'store': True, 'type': 'int32'}, {'facet': False, 'index': True, 'infix': False, 'locale': '', 'name': 'ratings_count', 'optional': False, 'sort': True, 'stem': False, 'stem_dictionary': '', 'store': True, 'type': 'int32'}, {'facet': False, 'index': True, 'infix': False, 'locale': '', 'name': 'average_rating', 'optional': False, 'sort': True, 'stem': Fal

In [5]:
# Importing all the data in the JSON file to the schema

with open('books.jsonl', 'r', encoding='utf-8') as jsonl_file:
    data = jsonl_file.read()
    client.collections['books'].documents.import_(data)

In [7]:
search_parameters={ # Searching the collection based on some parameter values
    'q':"harry potter",
    'query_by':"title,authors",
    'sort_by':"ratings_count:desc"
}

client.collections['books'].documents.search(search_parameters)

{'facet_counts': [],
 'found': 17,
 'hits': [{'document': {'authors': ['J.K. Rowling', ' Mary GrandPré'],
    'average_rating': 4.44,
    'id': '2',
    'image_url': 'https://images.gr-assets.com/books/1474154022m/3.jpg',
    'publication_year': 1997,
    'ratings_count': 4602479,
    'title': "Harry Potter and the Philosopher's Stone"},
   'highlight': {'title': {'matched_tokens': ['Harry', 'Potter'],
     'snippet': "<mark>Harry</mark> <mark>Potter</mark> and the Philosopher's Stone"}},
   'highlights': [{'field': 'title',
     'matched_tokens': ['Harry', 'Potter'],
     'snippet': "<mark>Harry</mark> <mark>Potter</mark> and the Philosopher's Stone"}],
   'text_match': 1157451471441100921,
   'text_match_info': {'best_field_score': '2211897868288',
    'best_field_weight': 15,
    'fields_matched': 1,
    'num_tokens_dropped': 0,
    'score': '1157451471441100921',
    'tokens_matched': 2,
    'typo_prefix_score': 0}},
  {'document': {'authors': ['J.K. Rowling', ' Mary GrandPré', ' R

In [8]:
# Lets put some filters (Like we do in SQL - the "WHERE" clause)

search_parameters = {
  'q'         : 'harry potter',
  'query_by'  : 'title',
  'filter_by' : 'publication_year:<1998',
  'sort_by'   : 'publication_year:desc'
}

client.collections['books'].documents.search(search_parameters)

{'facet_counts': [],
 'found': 1,
 'hits': [{'document': {'authors': ['J.K. Rowling', ' Mary GrandPré'],
    'average_rating': 4.44,
    'id': '2',
    'image_url': 'https://images.gr-assets.com/books/1474154022m/3.jpg',
    'publication_year': 1997,
    'ratings_count': 4602479,
    'title': "Harry Potter and the Philosopher's Stone"},
   'highlight': {'title': {'matched_tokens': ['Harry', 'Potter'],
     'snippet': "<mark>Harry</mark> <mark>Potter</mark> and the Philosopher's Stone"}},
   'highlights': [{'field': 'title',
     'matched_tokens': ['Harry', 'Potter'],
     'snippet': "<mark>Harry</mark> <mark>Potter</mark> and the Philosopher's Stone"}],
   'text_match': 1157451471441100921,
   'text_match_info': {'best_field_score': '2211897868288',
    'best_field_weight': 15,
    'fields_matched': 1,
    'num_tokens_dropped': 0,
    'score': '1157451471441100921',
    'tokens_matched': 2,
    'typo_prefix_score': 0}}],
 'out_of': 9979,
 'page': 1,
 'request_params': {'collection_name

In [10]:
search_parameters = {
  'q'         : 'experyment',
  'query_by'  : 'title',
  'facet_by'  : 'authors',
  'sort_by'   : 'average_rating:desc'
}

client.collections['books'].documents.search(search_parameters)

{'facet_counts': [{'counts': [{'count': 1,
     'highlighted': ' Käthe Mazur',
     'value': ' Käthe Mazur'},
    {'count': 1, 'highlighted': 'Mahatma Gandhi', 'value': 'Mahatma Gandhi'},
    {'count': 1, 'highlighted': 'Gretchen Rubin', 'value': 'Gretchen Rubin'},
    {'count': 1,
     'highlighted': 'James Patterson',
     'value': 'James Patterson'}],
   'field_name': 'authors',
   'sampled': False,
   'stats': {'total_values': 4}}],
 'found': 3,
 'hits': [{'document': {'authors': ['James Patterson'],
    'average_rating': 4.08,
    'id': '569',
    'image_url': 'https://images.gr-assets.com/books/1339277875m/13152.jpg',
    'publication_year': 2005,
    'ratings_count': 172302,
    'title': 'The Angel Experiment'},
   'highlight': {'title': {'matched_tokens': ['Experiment'],
     'snippet': 'The Angel <mark>Experiment</mark>'}},
   'highlights': [{'field': 'title',
     'matched_tokens': ['Experiment'],
     'snippet': 'The Angel <mark>Experiment</mark>'}],
   'text_match': 5787300

<br>
<br>

## **Langchain + Typsense + Groq LLM + RAG Application**

In [11]:
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import Typesense
from langchain_text_splitters import CharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_groq import ChatGroq

/Users/soubhiksinha/GitHub/RAG-Basics/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [14]:
import os
from dotenv import load_dotenv
load_dotenv()

os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")

In [17]:
# Text Spitting

loader = TextLoader("test.txt")
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
docs = text_splitter.split_documents(documents)

embeddings = HuggingFaceEmbeddings()

/var/folders/mb/n07mq38j10x93x8ln22q2h180000gn/T/ipykernel_41804/3420304569.py:8: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings()
/var/folders/mb/n07mq38j10x93x8ln22q2h180000gn/T/ipykernel_41804/3420304569.py:8: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  embeddings = HuggingFaceEmbeddings()


In [ ]:
docsearch=Typesense.from_documents(
    docs,
    embeddings,
    typesense_client_params={
        'host': '7k2vp1z3qrhta4x5p-1.a1.typesense.net',
        'port': '443',
        'protocol': 'https',
        'typesense_api_key': 'YOUR_TYPESENSE_API_KEY', # The Admin API (and not the READ-ONLY API)
        'typesense_collection_name': "LangChain-RAG-TypeSense"
    },
)

In [21]:
# Similarity Search

query = "What is a Transformer ?"
found_docs = docsearch.similarity_search(query)
print(found_docs[0].page_content)

Transformers are now the foundation for state-of-the-art models in NLP, vision (ViT), speech, and multimodal AI. 
Their flexibility, scalability, and representational power make them the backbone of modern AI systems.

Attention Is All You Need (Vaswani et al., 2017) revolutionized the field of Natural Language Processing by 
proposing the Transformer architecture. The key insight was the use of self-attention mechanisms to replace recurrent 
and convolutional operations, thereby allowing parallelization and better handling of long-range dependencies.

The Transformer model consists of an encoder-decoder structure with multi-head self-attention, position-wise feedforward 
networks, residual connections, and positional encodings. The self-attention mechanism computes attention scores using 
the scaled dot-product between queries, keys, and values, enabling the model to weigh contextual relevance dynamically.


In [22]:
# Converting into a RETRIEVER

retriever = docsearch.as_retriever()
retriever

VectorStoreRetriever(tags=['Typesense', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.typesense.Typesense object at 0x157364ec0>, search_kwargs={})

In [23]:
query = "What is a Attention ?"
retriever.invoke(query)[0]

Document(metadata={'source': 'test.txt'}, page_content='Attention Is All You Need (Vaswani et al., 2017) revolutionized the field of Natural Language Processing by \nproposing the Transformer architecture. The key insight was the use of self-attention mechanisms to replace recurrent \nand convolutional operations, thereby allowing parallelization and better handling of long-range dependencies.\n\nThe Transformer model consists of an encoder-decoder structure with multi-head self-attention, position-wise feedforward \nnetworks, residual connections, and positional encodings. The self-attention mechanism computes attention scores using \nthe scaled dot-product between queries, keys, and values, enabling the model to weigh contextual relevance dynamically.')